In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# # 大庆市
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
region = 'daqing'
sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'
# region = 'prd'
# sample_region = 'PRD'
# region = 'tibet'
# sample_region = 'TIBET'

In [5]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

In [6]:
# def Compare(feature):
#     cluster = ee.Number(feature.get('cluster'))
#     waterclass = ee.Number(feature.get('waterclass'))
#     ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
#     return ft
# # k-mean聚类
# def k_mean(sampleSET):
#     clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
#     result = sampleSET.cluster(clusterer)
#     right = result.map(Compare).filter(ee.Filter.eq('eq',0))
#     error = result.map(Compare).filter(ee.Filter.eq('eq',1))
#     filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
#     return filtered_sample

def classified_image(img):  
    image = maskSR(img).select(bands).classify(trainedClassifier).eq(1).remap([0,1],[1,2]).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 500,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 500,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return class_image.set({'system:id':img.get('system:id')}).set({'CLOUD_COVER':img.get('CLOUD_COVER')}).set({'invalid_percentage':rate})

def waterArea(image):
    classified_image = image.select('waterclass').eq(2).rename('waterclass')
    water_area = classified_image.multiply(ee.Image.pixelArea()).divide(1e6)
    waterarea = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 200,
        'maxPixels': 1e14,
        'tileScale': 2,
    })
    return image.set({'waterarea': waterarea.get('waterclass')})

# img指经过指数计算，但未做云掩膜的图像
def occurrence_Histogram(class_image):
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': roi,
        'scale': 30,
        'bestEffort': True,
        'tileScale': 2,
    })
    return class_image.set({'occurrence_HistogramCount': occurrence_HistogramCount.get('occurrence')})

def AutomaticCorrection_threshold(class_image):
    histogram = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('bucketMeans'))
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    return class_image.set({'occurrence_threshold':occurrence_threshold})

# img指经过指数计算(需要校正的图像)，但未做云掩膜的图像
def AutomaticCorrection_enhanced(class_image):
    basemap = ee.Image.constant(0).toFloat().updateMask(class_image.gte(0)).rename('waterclass')
    water = class_image.eq(2).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_no_data = occurrence.updateMask(class_image.eq(0).selfMask())
    occurrence_threshold = class_image.get('occurrence_threshold')
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

# 5-year interval

In [7]:
# TIME = [['2000','0105'],['2005','0105'],['2010','0610'],['2015','1115'],['2020','1620']],[['2000'],['2005'],['2010'],['2015'],['2020']]
TIME = [['2000'],['2005'],['2010'],['2015'],['2020']]
# TIME = [['2015']]
for time in TIME:
    print(time[0])
    startDate = time[0] + '-01-01'
    endDate = time[0] + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    landsat_image = l8.merge(l7).merge(l5).map(water_index)
    # print(landsat_image.size().getInfo())
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    # 'TIBET','tibet' 需要修改
    # filtered_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_' + time[1] + '_v2')
    filtered_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920_v2')
    trainedClassifier = ee.Classifier.smileRandomForest(150).train(filtered_samples,label,bands)
    landsat_images = landsat_image.map(classified_image).filter(ee.Filter.gte('invalid_percentage',5)).filter(ee.Filter.lt('invalid_percentage',95))
    No_correct_image5 = landsat_image.map(classified_image).filter(ee.Filter.lt('invalid_percentage',5))
    print('invalid_percentage: ',No_correct_image5.size().getInfo())
    No_correct_image6 = landsat_image.map(classified_image).filter(ee.Filter.gte('invalid_percentage',95))
#     # 两步筛选
#     probable_correct_image = landsat_images.map(waterArea).filter(ee.Filter.gt('waterarea',5))
#     No_correct_image1 = landsat_images.map(waterArea).filter(ee.Filter.lte('waterarea',5))
#     occurrence_threshold = probable_correct_image.map(occurrence_Histogram).filter(ee.Filter.neq('occurrence_HistogramCount',None)).map(AutomaticCorrection_threshold)
#     correct_image = occurrence_threshold.filter(ee.Filter.gt('occurrence_threshold',5)).filter(ee.Filter.lte('occurrence_threshold',75))
#     No_correct_image2 = occurrence_threshold.filter(ee.Filter.lte('occurrence_threshold',5))
#     No_correct_image3 = occurrence_threshold.filter(ee.Filter.gt('occurrence_threshold',75))
#     No_correct_image4 = probable_correct_image.map(occurrence_Histogram).filter(ee.Filter.eq('occurrence_HistogramCount',None))
#     correct_image = correct_image
#     print('correct: ',correct_image.size().getInfo())
#     No_correct_image = No_correct_image1.merge(No_correct_image2).merge(No_correct_image3).merge(No_correct_image4).merge(No_correct_image5).merge(No_correct_image6)
#     # print('nocorrect: ',No_correct_image.size().getInfo())

2000
invalid_percentage:  31
2005
invalid_percentage:  29
2010
invalid_percentage:  43
2015
invalid_percentage:  35
2020
invalid_percentage:  26


In [ ]:
print(correct_image.aggregate_array('system:id').getInfo())

In [ ]:
['LANDSAT/LC08/C01/T1_SR/LC08_138035_20150628',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20150714',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20150730',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20150815',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20150831',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20150916',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20151018',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20151119',
 'LANDSAT/LC08/C01/T1_SR/LC08_138035_20151205',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150511',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150628',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150714',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150730',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150815',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150831',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20150916',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20151018',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20151103',
 'LANDSAT/LC08/C01/T1_SR/LC08_138036_20151119',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150110',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150126',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150227',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150315',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150416',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150518',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150603',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150619',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150705',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150806',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150822',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150907',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20150923',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20151009',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20151025',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20151110',
 'LANDSAT/LC08/C01/T1_SR/LC08_139035_20151126',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150110',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150126',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150227',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150315',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150416',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150518',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150603',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150619',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150822',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150907',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20150923',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20151009',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20151025',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20151110'
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20151126',
 'LANDSAT/LC08/C01/T1_SR/LC08_139036_20151212',
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20150525', 
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20150626', 
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20150712', 
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20150728', 
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20150813', 
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20150914',
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20151016',
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20151101',
 'LANDSAT/LC08/C01/T1_SR/LC08_140035_20151203',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20150525',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20150712',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20150728', 
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20150813',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20151016',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20151101',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20151117',
 'LANDSAT/LC08/C01/T1_SR/LC08_140036_20151203', 
 'LANDSAT/LE07/C01/T1_SR/LE07_138035_20150212',
 'LANDSAT/LE07/C01/T1_SR/LE07_138035_20150316',
 'LANDSAT/LE07/C01/T1_SR/LE07_138035_20150401',
 'LANDSAT/LE07/C01/T1_SR/LE07_138035_20150706',
 'LANDSAT/LE07/C01/T1_SR/LE07_138035_20150807',
 'LANDSAT/LE07/C01/T1_SR/LE07_138036_20150620',
 'LANDSAT/LE07/C01/T1_SR/LE07_138036_20150706',
 'LANDSAT/LE07/C01/T1_SR/LE07_138036_20150722',
 'LANDSAT/LE07/C01/T1_SR/LE07_138036_20150807',
 'LANDSAT/LE07/C01/T1_SR/LE07_138036_20150908',
 'LANDSAT/LE07/C01/T1_SR/LE07_138036_20151026', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150102', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150203', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150323',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150424',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150510',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150526',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150627',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150713',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150729',
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150814', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150830', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20150915', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20151017', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20151118', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20151204', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139035_20151220',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150102', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150118', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150203',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150307',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150424',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150510',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150526', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150611', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150627', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150713', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150729', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150814', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150830', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20150915', 
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20151017',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20151102',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20151118',
 'LANDSAT/LE07/C01/T1_SR/LE07_139036_20151220',
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150415', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150517', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150618', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150704', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150720', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150821', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20150922', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20151008', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20151024', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20151109', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20151125', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140035_20151211', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20150517', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20150618', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20150720', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20150821', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20150922', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20151008', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20151024', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20151109', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20151125', 
 'LANDSAT/LE07/C01/T1_SR/LE07_140036_20151211']

In [ ]:
## tibet
2000
144
correct:  11
nocorrect:  133
2005
163
correct:  44
nocorrect:  119
2010
180
correct:  90
nocorrect:  90
2015
259
correct:  136
nocorrect:  123
2020
244
correct:  48
nocorrect:  196

# Proportion of corrected images

In [11]:
print('A')
print(182+142+131+157+139)
print(30+29+21+44+3)
print((30+29+21+44+3)/(182+142+131+157+139))

print('B')
print(205+194+175+223+219)
print(2+3+1+2+0)
print((2+3+1+2+0)/(205+194+175+223+219))

print('C')
print(70+76+69+110+85)
print(4+10+3+15+2)
print((4+10+3+15+2)/(70+76+69+110+85))

print('D')
print(141+155+89+153+137)
print(20+13+16+15+23)
print((20+13+16+15+23)/(141+155+89+153+137))

print('E')
print(144+163+180+259+244)
print(11+44+90+136+48)
print((11+44+90+136+48)/(144+163+180+259+244))

A
751
127
0.16910785619174434
B
1016
8
0.007874015748031496
C
410
34
0.08292682926829269
D
675
87
0.1288888888888889
E
990
329
0.3323232323232323


In [13]:
print('A')
print(31+29+43+35+26)
print(30+29+21+44+3)
print(31+29+43+35+26+30+29+21+44+3)
print((30+29+21+44+30)/(31+29+43+35+26+30+29+21+44+3))

print('B')
print(74+79+68+78+87)
print(2+3+1+2+0)
print(74+79+68+78+87+2+3+1+2+0)
print((2+3+1+2+0)/(74+79+68+78+87+2+3+1+2+0))

print('C')
print(18+17+18+25+24)
print(4+10+3+15+2)
print(18+17+18+25+24+4+10+3+15+2)
print((4+10+3+15+2)/(18+17+18+25+24+4+10+3+15+2))

print('D')
print(19+13+17+24+11)
print(20+13+16+15+23)
print(19+13+17+24+11+20+13+16+15+23)
print((20+13+16+15+23)/(19+13+17+24+11+20+13+16+15+23))

print('E')
print(5+6+6+17+31)
print(11+44+90+136+48)
print(5+6+6+17+31+11+44+90+136+48)
print((11+44+90+136+48)/(5+6+6+17+31+11+44+90+136+48))

A
164
127
291
0.5292096219931272
B
386
8
394
0.02030456852791878
C
102
34
136
0.25
D
84
87
171
0.5087719298245614
E
65
329
394
0.8350253807106599
